# CV Routine

In [1]:
%matplotlib inline
# Load all necessary packages
import sys

import pandas as pd

sys.path.append("../")
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.inprocessing.fairness_adjuster import FairnessAdjuster
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import (
    load_preproc_data_adult,
    load_preproc_data_compas,
    load_preproc_data_german,
)
from aif360.datasets import (
    AdultDataset,
    BinaryLabelDataset,
    CompasDataset,
    GermanDataset,
)
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from IPython.display import Markdown, display
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

tf.disable_eager_execution()

2024-12-10 00:55:19.469913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733792119.481472   96779 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733792119.484969   96779 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 00:55:19.497534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/aif360/lib/python3.11/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated

In [2]:
dataset_orig = load_preproc_data_german(["age", "sex"])
privileged_groups = [{"age": 1}]
unprivileged_groups = [{"age": 0}]

/home/ubuntu/AIF360_dev/aif360/algorithms/preprocessing/optim_preproc_helpers/data_preproc_functions.py:261: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['personal_status'].replace(status_map)


In [3]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate

min_max_scaler = MaxAbsScaler()

Prep

In [4]:
dataset_orig.features = min_max_scaler.fit_transform(dataset_orig.features)

CV

In [5]:
def get_metrics(
    dataset_orig_test, dataset_pred_test, unprivileged_groups, privileged_groups
):
    metric_dataset_debiasing_test = BinaryLabelDatasetMetric(
        dataset_orig_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
    )
    classified_metric_debiasing_test = ClassificationMetric(
        dataset_orig_test,
        dataset_pred_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
    )
    metrics_dict = {
        "Test Set: mean outcomes difference": metric_dataset_debiasing_test.mean_difference(),
        "Test Set: Classification accuracy": classified_metric_debiasing_test.accuracy(),
        "Test Set: Disparate impact": classified_metric_debiasing_test.disparate_impact(),
        "Test Set: Average odds difference": classified_metric_debiasing_test.average_odds_difference(),
    }
    return metrics_dict

In [6]:
from copy import copy, deepcopy

import numpy as np
from sklearn.metrics import accuracy_score  # Replace with desired metric
from sklearn.model_selection import KFold


def cross_validate_seeds(
    model_class,
    model_kwargs,
    dataset,
    seeds=5,
    n_folds=5,
):
    """
    Runs cross-validation with multiple seeds.

    Parameters:
        model: The Scikit-learn model to validate.
        X: Feature matrix.
        y: Target vector.
        seeds: Number of seeds for cross-validation.
        folds: Number of folds for each cross-validation.

    Returns:
        results: Dictionary containing accuracy scores for each seed and fold.
    """
    results = {}
    scope_name = model_kwargs["scope_name"]
    original_df = dataset.convert_to_dataframe()[0]
    for seed in range(seeds):
        print(f"\nSeed {seed + 1}/{seeds}")
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        seed_results = {}
        for to_debias in [True, False]:
            model_kwargs["debias"] = to_debias
            fold_results = []
            for fold, (train_idx, test_idx) in enumerate(kf.split(original_df)):
                print(f"{seed=} || {fold=}")
                # Splitting data
                dataset_X_train, dataset_X_test = dataset.subset(
                    train_idx
                ), dataset.subset(test_idx)

                # Training model
                sess = tf.Session()
                model_kwargs["sess"] = sess
                model_kwargs["scope_name"] = scope_name + f"_{seed}_{fold}"
                model = model_class(**model_kwargs)
                model.fit(dataset_X_train, verbose=False)

                # Making predictions
                dataset_preds = model.predict(dataset_X_test)

                # Evaluating model
                metrics = get_metrics(
                    dataset_X_test,
                    dataset_preds,
                    model_kwargs["unprivileged_groups"],
                    model_kwargs["privileged_groups"],
                )
                fold_results.append(metrics)

                # end session
                sess.close()
                tf.reset_default_graph()
            seed_results[f"{to_debias=}"] = fold_results
        results[f"Seed {seed}"] = seed_results

    return results

Fairness adjuster

In [7]:
model_class = FairnessAdjuster
model_kwargs = {
    "privileged_groups": privileged_groups,
    "unprivileged_groups": unprivileged_groups,
    "scope_name": "debiased_classifier",
    "adversary_loss_weight": 0.01,
    "classifier_num_hidden_units": 100,
}

In [8]:
results = cross_validate_seeds(
    model_class,
    model_kwargs,
    dataset_orig,
    seeds=3,
    n_folds=10,
)


Seed 1/3
seed=0 || fold=0
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-12-10 00:55:23.699157: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1733792123.816294   96779 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-12-10 00:55:25.151927: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier_0_0/debiased_classifier_0_0/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier_0_0/debiased_classifier_0_0/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier_0_0/debiased_classifier_0_0/classifier_model/b2/Adam_1, debiased_classifier_0_0/debiased_classifier_0_0/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the 

seed=0 || fold=1


2024-12-10 00:55:26.805433: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier_0_1/debiased_classifier_0_1/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier_0_1/debiased_classifier_0_1/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier_0_1/debiased_classifier_0_1/classifier_model/b2/Adam_1, debiased_classifier_0_1/debiased_classifier_0_1/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


seed=0 || fold=2


...

2024-12-10 00:56:46.329395: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier_2_7/debiased_classifier_2_7/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier_2_7/debiased_classifier_2_7/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier_2_7/debiased_classifier_2_7/classifier_model/b2/Adam_1, debiased_classifier_2_7/debiased_classifier_2_7/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


seed=2 || fold=8


2024-12-10 00:56:47.565429: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier_2_8/debiased_classifier_2_8/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier_2_8/debiased_classifier_2_8/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier_2_8/debiased_classifier_2_8/classifier_model/b2/Adam_1, debiased_classifier_2_8/debiased_classifier_2_8/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


seed=2 || fold=9


2024-12-10 00:56:48.813170: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier_2_9/debiased_classifier_2_9/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier_2_9/debiased_classifier_2_9/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier_2_9/debiased_classifier_2_9/classifier_model/b2/Adam_1, debiased_classifier_2_9/debiased_classifier_2_9/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [9]:
import pickle

with open("./data/adjuster_group_cv_results.pickle", "wb") as path:
    pickle.dump(results, path, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
aggs = {}
for seed, res_dict in results.items():
    aggs[seed] = {}
    for to_debias, fold_res_dict in res_dict.items():
        res = pd.DataFrame(fold_res_dict).quantile([0.025, 0.975]).T
        res["means"] = pd.DataFrame(fold_res_dict).mean()
        aggs[seed][to_debias] = res.apply(
            lambda x: f"{x.means:.4f} ({x[0.025]:.4f},{x[0.975]:.4f})", axis=1
        )

reform = {
    (outerKey, innerKey): values
    for outerKey, innerDict in aggs.items()
    for innerKey, values in innerDict.items()
}
pd.DataFrame(reform).T

Test Set: mean outcomes difference  \
Seed 0 to_debias=True            -0.1388 (-0.2802,0.0504)   
       to_debias=False           -0.1388 (-0.2802,0.0504)   
Seed 1 to_debias=True           -0.1554 (-0.3321,-0.0244)   
       to_debias=False          -0.1554 (-0.3321,-0.0244)   
Seed 2 to_debias=True           -0.1534 (-0.3167,-0.0148)   
       to_debias=False          -0.1534 (-0.3167,-0.0148)   

                       Test Set: Classification accuracy  \
Seed 0 to_debias=True             0.7130 (0.6268,0.7678)   
       to_debias=False            0.7120 (0.6268,0.7678)   
Seed 1 to_debias=True             0.7070 (0.6312,0.7600)   
       to_debias=False            0.7090 (0.6545,0.7600)   
Seed 2 to_debias=True             0.7030 (0.6500,0.7655)   
       to_debias=False            0.7080 (0.6545,0.7855)   

                       Test Set: Disparate impact  \
Seed 0 to_debias=True      0.7423 (0.6229,0.9049)   
       to_debias=False     0.7411 (0.6229,0.8960)   
Seed 1 to_debias=True      0.7062 (0.3700,0.9553)   
       to_debias=False     0.7376 (0.6111,0.9259)   
Seed 2 to_debias=True      0.7133 (0.5000,0.9403)   
       to_debias=False     0.7493 (0.6141,0.8750)   

                       Test Set: Average odds difference  
Seed 0 to_debias=True          -0.2852 (-0.4432,-0.1024)  
       to_debias=False         -0.2882 (-0.4432,-0.1252)  
Seed 1 to_debias=True          -0.3154 (-0.6683,-0.0551)  
       to_debias=False         -0.2882 (-0.4803,-0.0913)  
Seed 2 to_debias=True          -0.3023 (-0.5223,-0.0556)  
       to_debias=False         -0.2653 (-0.4187,-0.1483)

AD

In [11]:
model_class = AdversarialDebiasing
model_kwargs = {
    "privileged_groups": privileged_groups,
    "unprivileged_groups": unprivileged_groups,
    "scope_name": "adversarial_debiased_classifier",
    "adversary_loss_weight": 0.01,
    "classifier_num_hidden_units": 100,
}

In [12]:
results = cross_validate_seeds(
    model_class,
    model_kwargs,
    dataset_orig,
    seeds=3,
    n_folds=10,
)


Seed 1/3
seed=0 || fold=0
seed=0 || fold=1
seed=0 || fold=2
seed=0 || fold=3
seed=0 || fold=4
seed=0 || fold=5
seed=0 || fold=6
seed=0 || fold=7
seed=0 || fold=8
seed=0 || fold=9
seed=0 || fold=0
seed=0 || fold=1
seed=0 || fold=2
seed=0 || fold=3
seed=0 || fold=4
seed=0 || fold=5
seed=0 || fold=6
seed=0 || fold=7
seed=0 || fold=8
seed=0 || fold=9

Seed 2/3
seed=1 || fold=0
seed=1 || fold=1
seed=1 || fold=2
seed=1 || fold=3
seed=1 || fold=4
seed=1 || fold=5
seed=1 || fold=6
seed=1 || fold=7
seed=1 || fold=8
seed=1 || fold=9
seed=1 || fold=0
seed=1 || fold=1
seed=1 || fold=2
seed=1 || fold=3
seed=1 || fold=4
seed=1 || fold=5
seed=1 || fold=6
seed=1 || fold=7
seed=1 || fold=8
seed=1 || fold=9

Seed 3/3
seed=2 || fold=0
seed=2 || fold=1
seed=2 || fold=2
seed=2 || fold=3
seed=2 || fold=4
seed=2 || fold=5
seed=2 || fold=6
seed=2 || fold=7
seed=2 || fold=8
seed=2 || fold=9
seed=2 || fold=0
seed=2 || fold=1
seed=2 || fold=2
seed=2 || fold=3
seed=2 || fold=4
seed=2 || fold=5
seed=2 || fold=6
s

In [13]:
import pickle

with open("./data/ad_group_cv_results.pickle", "wb") as path:
    pickle.dump(results, path, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
aggs = {}
for seed, res_dict in results.items():
    aggs[seed] = {}
    for to_debias, fold_res_dict in res_dict.items():
        res = pd.DataFrame(fold_res_dict).quantile([0.025, 0.975]).T
        res["means"] = pd.DataFrame(fold_res_dict).mean()
        aggs[seed][to_debias] = res.apply(
            lambda x: f"{x.means:.4f} ({x[0.025]:.4f},{x[0.975]:.4f})", axis=1
        )

reform = {
    (outerKey, innerKey): values
    for outerKey, innerDict in aggs.items()
    for innerKey, values in innerDict.items()
}
pd.DataFrame(reform).T

Test Set: mean outcomes difference  \
Seed 0 to_debias=True            -0.1388 (-0.2802,0.0504)   
       to_debias=False           -0.1388 (-0.2802,0.0504)   
Seed 1 to_debias=True           -0.1554 (-0.3321,-0.0244)   
       to_debias=False          -0.1554 (-0.3321,-0.0244)   
Seed 2 to_debias=True           -0.1534 (-0.3167,-0.0148)   
       to_debias=False          -0.1534 (-0.3167,-0.0148)   

                       Test Set: Classification accuracy  \
Seed 0 to_debias=True             0.6740 (0.5867,0.7610)   
       to_debias=False            0.7070 (0.6268,0.7755)   
Seed 1 to_debias=True             0.6740 (0.5912,0.7355)   
       to_debias=False            0.7090 (0.6545,0.7600)   
Seed 2 to_debias=True             0.6590 (0.5745,0.7500)   
       to_debias=False            0.7080 (0.6545,0.7878)   

                       Test Set: Disparate impact  \
Seed 0 to_debias=True      1.0463 (0.7028,1.3873)   
       to_debias=False     0.7304 (0.6279,0.8873)   
Seed 1 to_debias=True      0.9905 (0.6111,1.3181)   
       to_debias=False     0.7456 (0.6111,0.9364)   
Seed 2 to_debias=True      1.0549 (0.5482,1.8830)   
       to_debias=False     0.7394 (0.6082,0.8750)   

                       Test Set: Average odds difference  
Seed 0 to_debias=True            0.0062 (-0.3203,0.2919)  
       to_debias=False         -0.2906 (-0.4253,-0.1024)  
Seed 1 to_debias=True           -0.0357 (-0.4803,0.2489)  
       to_debias=False         -0.2790 (-0.4803,-0.0852)  
Seed 2 to_debias=True           -0.0199 (-0.4751,0.4999)  
       to_debias=False         -0.2744 (-0.4248,-0.1483)